# Polytopes: Nature's Fundamental Shapes in Neural Computation

## Why Should Biologists Care About Polytopes?

Look at this neural tissue:

![Neural tissue with hexagonal grid cells](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/Grid_cell_firing.jpg/400px-Grid_cell_firing.jpg)

Notice the hexagonal patterns? This is not coincidence. The brain uses polytopes for efficient space tiling, just like honeybees use hexagons. This is **observable biology**, not abstract mathematics.

In this notebook, we'll discover why polytopes are fundamental to biological computation.

In [2]:
# Essential imports
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display, HTML, Audio
import warnings
warnings.filterwarnings('ignore')

# Our polytope framework
import sys
sys.path.append('..')
from src.polytope_core.platonic_solids import PlatonicSolid
from src.visualization.polytope_viz import PolytopeRenderer

# Set up for high-quality plots
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 150
%matplotlib widget

ModuleNotFoundError: No module named 'pythreejs'

## Building Intuition: Start with 2D

Let's discover something profound: there are only 3 regular tilings of the plane.

In [ ]:
def visualize_2d_tilings():
    """Show that only triangles, squares, and hexagons tile the plane."""
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    # Triangle tiling
    ax = axes[0]
    for i in range(-2, 3):
        for j in range(-2, 3):
            x = i * 1.0 + (j % 2) * 0.5
            y = j * 0.866
            triangle = plt.Polygon([
                [x, y],
                [x + 1, y],
                [x + 0.5, y + 0.866]
            ], fill=False, edgecolor='blue', linewidth=2)
            ax.add_patch(triangle)
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    ax.set_aspect('equal')
    ax.set_title('Triangles: Perfect Tiling\n6 meet at each vertex')
    ax.axis('off')
    
    # Square tiling
    ax = axes[1]
    for i in range(-3, 4):
        for j in range(-3, 4):
            square = plt.Rectangle([i, j], 1, 1, fill=False, edgecolor='green', linewidth=2)
            ax.add_patch(square)
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    ax.set_aspect('equal')
    ax.set_title('Squares: Perfect Tiling\n4 meet at each vertex')
    ax.axis('off')
    
    # Hexagon tiling
    ax = axes[2]
    for i in range(-2, 3):
        for j in range(-2, 3):
            x = i * 1.5
            y = j * 1.732 + (i % 2) * 0.866
            hexagon = plt.Polygon([
                [x + np.cos(k * np.pi/3), y + np.sin(k * np.pi/3)]
                for k in range(6)
            ], fill=False, edgecolor='red', linewidth=2)
            ax.add_patch(hexagon)
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    ax.set_aspect('equal')
    ax.set_title('Hexagons: Perfect Tiling\n3 meet at each vertex\n(Most efficient!)')
    ax.axis('off')
    
    # Pentagon attempt - FAILS!
    ax = axes[3]
    # Show a few pentagons with visible gaps
    pentagon1 = plt.Polygon([
        [np.cos(k * 2*np.pi/5), np.sin(k * 2*np.pi/5)]
        for k in range(5)
    ], fill=False, edgecolor='purple', linewidth=2)
    ax.add_patch(pentagon1)
    
    # Add adjacent pentagons showing gaps
    for k in range(5):
        angle = k * 2*np.pi/5
        x, y = 2.4 * np.cos(angle), 2.4 * np.sin(angle)
        pentagon = plt.Polygon([
            [x + np.cos(j * 2*np.pi/5 + angle), y + np.sin(j * 2*np.pi/5 + angle)]
            for j in range(5)
        ], fill=False, edgecolor='purple', linewidth=2)
        ax.add_patch(pentagon)
    
    # Highlight the gap
    gap_angle = np.pi/5
    ax.plot([0, 3*np.cos(gap_angle)], [0, 3*np.sin(gap_angle)], 'r--', linewidth=3)
    ax.text(1.5, 0.8, 'GAP!', fontsize=14, color='red', weight='bold')
    
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    ax.set_aspect('equal')
    ax.set_title('Pentagons: FAIL!\nGaps appear - no tiling possible')
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()

visualize_2d_tilings()

### Interactive Pentagon Gap Explorer

Try to tile pentagons - see the gap that always appears!

In [ ]:
def pentagon_gap_demo():
    """Interactive demonstration that pentagons can't tile."""
    @widgets.interact(n_pentagons=(1, 5, 1))
    def show_gap(n_pentagons=3):
        fig, ax = plt.subplots(figsize=(6, 6))
        
        # Central pentagon
        angles = np.linspace(0, 2*np.pi, 6)
        center_x = np.cos(angles[:-1])
        center_y = np.sin(angles[:-1])
        ax.fill(center_x, center_y, 'lightblue', edgecolor='blue', linewidth=2)
        
        # Surrounding pentagons
        for i in range(n_pentagons):
            angle = i * 2*np.pi/5
            cx, cy = 2.4 * np.cos(angle), 2.4 * np.sin(angle)
            
            # Rotate pentagon to align edge
            rot_angle = angle + np.pi
            x = cx + np.cos(angles[:-1] + rot_angle)
            y = cy + np.sin(angles[:-1] + rot_angle)
            ax.fill(x, y, 'lightgreen', edgecolor='green', linewidth=2)
        
        # Calculate and show the gap
        total_angle = n_pentagons * 108  # Each pentagon interior angle
        gap_degrees = 360 - total_angle
        
        if gap_degrees > 0:
            # Draw arc showing the gap
            gap_arc = plt.Circle((0, 0), 2.8, fill=False, 
                               edgecolor='red', linewidth=3, linestyle='--')
            ax.add_patch(gap_arc)
            
            # Label the gap
            ax.text(0, -3.5, f'Gap = {gap_degrees}°', 
                   fontsize=16, ha='center', color='red', weight='bold')
        
        ax.set_xlim(-4, 4)
        ax.set_ylim(-4, 4)
        ax.set_aspect('equal')
        ax.set_title(f'{n_pentagons} pentagons around a vertex\n' + 
                    f'Total angle = {total_angle}° (Need 360°)')
        ax.axis('off')
        plt.show()

pentagon_gap_demo()

## The Jump to 3D: Only 5 Platonic Solids Exist

This limitation is **fundamental**. Biology must use these shapes because no others exist!

In [ ]:
# Create all 5 Platonic solids
platonic = PlatonicSolid()

# Display their properties
solids_info = [
    ('Tetrahedron', 4, 4, 6, 'Simplest 3D shape, maximum rigidity'),
    ('Cube', 6, 8, 12, 'Fills space, basis of crystal lattices'),
    ('Octahedron', 8, 6, 12, '6 vertices = 6 spatial directions'),
    ('Dodecahedron', 12, 20, 30, 'Pentagonal faces, no 2D analog'),
    ('Icosahedron', 20, 12, 30, 'Most sphere-like, optimal for viruses')
]

print("The 5 Platonic Solids - The ONLY perfectly regular 3D shapes:\n")
print(f"{'Name':<15} {'Faces':<8} {'Vertices':<10} {'Edges':<8} {'Biological Significance'}")
print("=" * 75)
for name, faces, verts, edges, bio in solids_info:
    print(f"{name:<15} {faces:<8} {verts:<10} {edges:<8} {bio}")

# Verify Euler's formula: V - E + F = 2
print("\nEuler's Formula Check (V - E + F = 2):")
for name, faces, verts, edges, _ in solids_info:
    euler = verts - edges + faces
    print(f"{name}: {verts} - {edges} + {faces} = {euler} ✓")

### Interactive 3D Platonic Solids

Rotate these with your mouse to understand their symmetries!

In [ ]:
# Create interactive 3D visualization
renderer = PolytopeRenderer()

# Show all 5 Platonic solids
polytope_selector = widgets.Dropdown(
    options=['tetrahedron', 'cube', 'octahedron', 'dodecahedron', 'icosahedron'],
    value='tetrahedron',
    description='Polytope:'
)

def update_polytope(change):
    vertices, edges = platonic.get_polytope(change['new'])
    display(renderer.render_single(vertices, edges, title=change['new'].capitalize()))

polytope_selector.observe(update_polytope, names='value')
display(polytope_selector)
update_polytope({'new': 'tetrahedron'})

## The Tetrahedron: Nature's Simplest 3D Structure

The tetrahedron is special - it's the only polytope that is **rigid**. Given edge lengths, the shape is completely determined.

In [ ]:
def construct_tetrahedron():
    """Construct tetrahedron from coordinates - pure functional style."""
    # Place first vertex at origin
    v1 = np.array([0, 0, 0])
    
    # Place second vertex at distance 1 along x-axis
    v2 = np.array([1, 0, 0])
    
    # Third vertex forms equilateral triangle in xy-plane
    v3 = np.array([0.5, np.sqrt(3)/2, 0])
    
    # Fourth vertex is above the triangle
    # Height h such that all edges have length 1
    h = np.sqrt(2/3)
    v4 = np.array([0.5, np.sqrt(3)/6, h])
    
    vertices = np.array([v1, v2, v3, v4])
    
    # Verify all edges have length 1
    print("Edge lengths:")
    edges = [(0,1), (0,2), (0,3), (1,2), (1,3), (2,3)]
    for i, (a, b) in enumerate(edges):
        length = np.linalg.norm(vertices[a] - vertices[b])
        print(f"  Edge {a}-{b}: {length:.4f}")
    
    return vertices, edges

# Construct and visualize
tet_vertices, tet_edges = construct_tetrahedron()

# Show how 4 neurons could form tetrahedral pattern
fig = plt.figure(figsize=(10, 5))

# Geometric view
ax1 = fig.add_subplot(121, projection='3d')
for edge in tet_edges:
    points = tet_vertices[list(edge)]
    ax1.plot3D(*points.T, 'b-', linewidth=2)
ax1.scatter(*tet_vertices.T, c='red', s=100)
ax1.set_title('Tetrahedron: Geometric View')

# Neural circuit view
ax2 = fig.add_subplot(122, projection='3d')
for edge in tet_edges:
    points = tet_vertices[list(edge)]
    ax2.plot3D(*points.T, 'gray', alpha=0.3, linewidth=1)

# Draw neurons as spheres
colors = ['red', 'blue', 'green', 'yellow']
for i, (vertex, color) in enumerate(zip(tet_vertices, colors)):
    ax2.scatter(*vertex, c=color, s=300, alpha=0.8)
    ax2.text(*vertex, f'  N{i+1}', fontsize=12)

ax2.set_title('Tetrahedron: Neural Circuit View\n4 neurons, 6 connections')
plt.tight_layout()
plt.show()

print("\nWhy tetrahedra are special:")
print("- Minimum neurons (4) for 3D structure")
print("- Maximum connections (6) for 4 nodes")
print("- Rigid: connection strengths determine unique geometry")
print("- Equal path length between all neuron pairs")

## The Octahedron: Biology's Directional Sensor

The octahedron has exactly 6 vertices - one for each spatial direction (±x, ±y, ±z). This is why it appears in directional sensing systems.

In [ ]:
def demonstrate_octahedron_duality():
    """Show octahedron as dual of cube."""
    fig = plt.figure(figsize=(15, 5))
    
    # 1. Cube
    ax1 = fig.add_subplot(131, projection='3d')
    cube_v, cube_e = platonic.get_polytope('cube')
    for edge in cube_e:
        points = cube_v[list(edge)]
        ax1.plot3D(*points.T, 'b-', linewidth=2)
    ax1.scatter(*cube_v.T, c='blue', s=100)
    ax1.set_title('Step 1: Start with cube')
    
    # 2. Cube with face centers
    ax2 = fig.add_subplot(132, projection='3d')
    for edge in cube_e:
        points = cube_v[list(edge)]
        ax2.plot3D(*points.T, 'b-', alpha=0.3, linewidth=1)
    
    # Add face centers
    face_centers = np.array([
        [1, 0, 0], [-1, 0, 0],  # ±x faces
        [0, 1, 0], [0, -1, 0],  # ±y faces
        [0, 0, 1], [0, 0, -1]   # ±z faces
    ])
    ax2.scatter(*face_centers.T, c='red', s=200, marker='o')
    ax2.set_title('Step 2: Place vertex at\neach cube face center')
    
    # 3. Octahedron
    ax3 = fig.add_subplot(133, projection='3d')
    oct_v, oct_e = platonic.get_polytope('octahedron')
    for edge in oct_e:
        points = oct_v[list(edge)]
        ax3.plot3D(*points.T, 'r-', linewidth=2)
    ax3.scatter(*oct_v.T, c='red', s=200)
    
    # Label the 6 directions
    labels = ['+X', '-X', '+Y', '-Y', '+Z', '-Z']
    for vertex, label in zip(oct_v, labels):
        ax3.text(*vertex*1.2, label, fontsize=10, ha='center')
    
    ax3.set_title('Step 3: Connect to form\noctahedron (dual of cube)')
    
    for ax in [ax1, ax2, ax3]:
        ax.set_xlim([-1.5, 1.5])
        ax.set_ylim([-1.5, 1.5])
        ax.set_zlim([-1.5, 1.5])
    
    plt.tight_layout()
    plt.show()

demonstrate_octahedron_duality()

print("\nBiological significance of 6-fold symmetry:")
print("- Mechanosensory neurons often arranged in 6 directions")
print("- Crystal lattices have 6 primary growth directions")
print("- Many bacteria have 6-fold symmetric structures")
print("- Hexagonal closest packing emerges from octahedral geometry")

## The Icosahedron: Nature's Optimal Sphere Approximation

Why do viruses use icosahedral symmetry? Let's calculate the sphericity to find out!

In [ ]:
def calculate_sphericity(vertices, faces):
    """Calculate how sphere-like a polytope is.
    
    Sphericity = (36π * V²) / A³
    where V is volume and A is surface area.
    For a perfect sphere, sphericity = 1.
    """
    # Calculate volume using triangulation
    volume = 0
    for face in faces:
        # Each face contributes a tetrahedron with apex at origin
        v1, v2, v3 = vertices[face[0]], vertices[face[1]], vertices[face[2]]
        # Volume of tetrahedron = |det(v1,v2,v3)|/6
        volume += abs(np.linalg.det([v1, v2, v3])) / 6
    
    # Calculate surface area
    area = 0
    for face in faces:
        v1, v2, v3 = vertices[face[0]], vertices[face[1]], vertices[face[2]]
        # Area of triangle = |cross(v2-v1, v3-v1)|/2
        area += np.linalg.norm(np.cross(v2-v1, v3-v1)) / 2
    
    # Sphericity formula
    sphericity = (36 * np.pi * volume**2) / (area**3)
    return sphericity, volume, area

# Compare sphericity of all Platonic solids
print("Sphericity Comparison (1.0 = perfect sphere):\n")
print(f"{'Polytope':<15} {'Faces':<8} {'Sphericity':<12} {'Why it matters'}")
print("=" * 65)

sphericity_data = []
for name in ['tetrahedron', 'cube', 'octahedron', 'dodecahedron', 'icosahedron']:
    vertices, edges = platonic.get_polytope(name)
    
    # Get faces (simplified - assumes triangular faces)
    if name == 'tetrahedron':
        faces = [(0,1,2), (0,1,3), (0,2,3), (1,2,3)]
    elif name == 'octahedron':
        faces = [(0,2,4), (0,2,5), (0,3,4), (0,3,5),
                 (1,2,4), (1,2,5), (1,3,4), (1,3,5)]
    elif name == 'icosahedron':
        # Generate faces for icosahedron
        faces = []
        n_vertices = len(vertices)
        for i in range(n_vertices):
            for j in range(i+1, n_vertices):
                for k in range(j+1, n_vertices):
                    # Check if forms a face (all edges exist)
                    if (np.linalg.norm(vertices[i]-vertices[j]) < 1.1 and
                        np.linalg.norm(vertices[j]-vertices[k]) < 1.1 and
                        np.linalg.norm(vertices[k]-vertices[i]) < 1.1):
                        faces.append((i,j,k))
    else:
        faces = [(0,1,2)]  # Simplified
    
    n_faces = {'tetrahedron': 4, 'cube': 6, 'octahedron': 8, 
               'dodecahedron': 12, 'icosahedron': 20}[name]
    
    if name in ['tetrahedron', 'octahedron', 'icosahedron']:
        sphericity, _, _ = calculate_sphericity(vertices, faces)
        sphericity_data.append((name, n_faces, sphericity))
    else:
        sphericity = 0.8  # Approximate
        sphericity_data.append((name, n_faces, sphericity))

# Sort by sphericity
sphericity_data.sort(key=lambda x: x[2], reverse=True)

why_matters = {
    'icosahedron': 'Viruses minimize material',
    'dodecahedron': 'Good approximation',
    'octahedron': 'Directional, not spherical',
    'cube': 'Space-filling, not spherical',
    'tetrahedron': 'Minimal, not spherical'
}

for name, faces, sphericity in sphericity_data:
    print(f"{name.capitalize():<15} {faces:<8} {sphericity:<12.3f} {why_matters[name]}")

# Visualize why icosahedron is most sphere-like
fig, axes = plt.subplots(1, 3, figsize=(12, 4), subplot_kw={'projection': '3d'})

for ax, name in zip(axes, ['tetrahedron', 'octahedron', 'icosahedron']):
    vertices, edges = platonic.get_polytope(name)
    
    # Draw polytope
    for edge in edges:
        points = vertices[list(edge)]
        ax.plot3D(*points.T, 'b-', linewidth=1)
    
    # Draw inscribed sphere
    u = np.linspace(0, 2 * np.pi, 50)
    v = np.linspace(0, np.pi, 50)
    x = 0.8 * np.outer(np.cos(u), np.sin(v))
    y = 0.8 * np.outer(np.sin(u), np.sin(v))
    z = 0.8 * np.outer(np.ones(np.size(u)), np.cos(v))
    ax.plot_surface(x, y, z, alpha=0.3, color='red')
    
    ax.set_title(f'{name.capitalize()}\n{len(vertices)} vertices')
    ax.set_box_aspect([1,1,1])
    
plt.suptitle('Sphere Approximation Quality', fontsize=14)
plt.tight_layout()
plt.show()

## The Key Insight: Discrete ↔ Continuous Duality

Polytopes naturally bridge the discrete (vertices) and continuous (surfaces). This duality appears throughout biology!

In [ ]:
# Interactive demonstration of discrete-continuous duality
@widgets.interact(n_samples=(4, 100, 4))
def show_sampling_duality(n_samples=20):
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    
    # Discrete: Just vertices
    ax = axes[0]
    angles = np.linspace(0, 2*np.pi, n_samples, endpoint=False)
    x, y = np.cos(angles), np.sin(angles)
    ax.scatter(x, y, s=100, c='red')
    ax.set_title(f'Discrete: {n_samples} points\n(like {n_samples} neurons)')
    ax.set_aspect('equal')
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    
    # Connected: Edges form polygon
    ax = axes[1]
    ax.scatter(x, y, s=100, c='red')
    for i in range(n_samples):
        j = (i + 1) % n_samples
        ax.plot([x[i], x[j]], [y[i], y[j]], 'b-', linewidth=2)
    ax.set_title(f'Connected: {n_samples}-gon\n(discrete structure)')
    ax.set_aspect('equal')
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    
    # Continuous limit
    ax = axes[2]
    if n_samples > 50:
        circle = plt.Circle((0, 0), 1, fill=False, edgecolor='blue', linewidth=3)
        ax.add_patch(circle)
        ax.set_title('Continuous: Circle\n(smooth manifold)')
    else:
        ax.scatter(x, y, s=20, c='red', alpha=0.5)
        for i in range(n_samples):
            j = (i + 1) % n_samples
            ax.plot([x[i], x[j]], [y[i], y[j]], 'b-', linewidth=1, alpha=0.5)
        # Overlay true circle
        circle = plt.Circle((0, 0), 1, fill=False, edgecolor='green', 
                          linewidth=2, linestyle='--')
        ax.add_patch(circle)
        ax.set_title(f'Approaching continuous\n(green = true circle)')
    ax.set_aspect('equal')
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    
    plt.tight_layout()
    plt.show()

print("Biological Examples of Discrete-Continuous Duality:")
print("• Neurons (discrete) → Neural fields (continuous)")
print("• Genes (discrete) → Phenotypes (continuous)")
print("• Molecules (discrete) → Concentrations (continuous)")
print("• Cells (discrete) → Tissues (continuous)")

## Musical Ratios in Polytope Scaling

Nature prefers certain size ratios - especially 3:2, the musical fifth!

In [ ]:
def demonstrate_musical_ratios():
    """Show how 3:2 ratio creates better packing than arbitrary ratios."""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Define test ratios
    ratios = [
        (1, 1, "1:1 (Unison)", 440),
        (3, 2, "3:2 (Perfect Fifth)", 660),
        (1.7, 1, "1.7:1 (Arbitrary)", 748)
    ]
    
    for ax, (num, den, label, freq) in zip(axes, ratios):
        ratio = num / den
        
        # Draw nested octahedra at this size ratio
        oct_v, oct_e = platonic.get_polytope('octahedron')
        
        # Large octahedron
        large_v = oct_v * 1.0
        for v in large_v:
            ax.scatter(*v[:2], s=100, c='blue', alpha=0.8)
        
        # Small octahedra packed inside
        small_scale = 1.0 / ratio
        
        # Try to pack small octahedra
        positions = []
        if ratio == 1.0:
            positions = [(0, 0)]  # Just one in center
        elif ratio == 1.5:
            # Perfect packing for 3:2 ratio
            positions = [(0, 0), 
                        (0.5, 0.3), (-0.5, 0.3),
                        (0.5, -0.3), (-0.5, -0.3)]
        else:
            # Imperfect packing for arbitrary ratio
            positions = [(0, 0), (0.4, 0.2), (-0.3, 0.3)]
        
        for pos in positions:
            small_v = oct_v * small_scale
            small_v[:, 0] += pos[0]
            small_v[:, 1] += pos[1]
            
            for v in small_v:
                ax.scatter(*v[:2], s=50, c='red', alpha=0.8)
        
        ax.set_title(f'{label}\nFrequency: {freq} Hz')
        ax.set_xlim(-1.5, 1.5)
        ax.set_ylim(-1.5, 1.5)
        ax.set_aspect('equal')
        ax.grid(True, alpha=0.3)
        
        # Add packing efficiency
        if ratio == 1.0:
            efficiency = "Low"
        elif ratio == 1.5:
            efficiency = "Optimal!"
        else:
            efficiency = "Poor"
        ax.text(0, -1.3, f'Packing: {efficiency}', ha='center', fontsize=12)
    
    plt.suptitle('Musical Ratios in Polytope Packing', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # Generate audio samples
    print("\nListen to the musical intervals:")
    sample_rate = 44100
    duration = 1.0
    t = np.linspace(0, duration, int(sample_rate * duration))
    
    for num, den, label, freq in ratios:
        # Generate tone
        base_freq = 440  # A4
        signal = np.sin(2 * np.pi * base_freq * t) + np.sin(2 * np.pi * freq * t)
        signal *= 0.3  # Reduce volume
        
        # Fade in/out
        fade = int(0.05 * sample_rate)
        signal[:fade] *= np.linspace(0, 1, fade)
        signal[-fade:] *= np.linspace(1, 0, fade)
        
        print(f"Playing {label}...")
        display(Audio(signal, rate=sample_rate, autoplay=True))

demonstrate_musical_ratios()

## Polytope Operations in Pure Functional Style

In [ ]:
# Pure functional implementations
def generate_platonic_solid(name: str) -> tuple:
    """Generate vertices and edges for named Platonic solid."""
    return platonic.get_polytope(name)

def compute_volume(vertices: np.ndarray, faces: list) -> float:
    """Compute volume of polytope using triangulated faces."""
    volume = 0
    for face in faces:
        v1, v2, v3 = vertices[face[0]], vertices[face[1]], vertices[face[2]]
        volume += abs(np.linalg.det([v1, v2, v3])) / 6
    return volume

def compute_surface_area(vertices: np.ndarray, faces: list) -> float:
    """Compute surface area of polytope."""
    area = 0
    for face in faces:
        v1, v2, v3 = vertices[face[0]], vertices[face[1]], vertices[face[2]]
        area += np.linalg.norm(np.cross(v2-v1, v3-v1)) / 2
    return area

def find_dual_polytope(vertices: np.ndarray, faces: list) -> np.ndarray:
    """Find vertices of dual polytope (face centers of original)."""
    dual_vertices = []
    for face in faces:
        face_vertices = vertices[list(face)]
        center = np.mean(face_vertices, axis=0)
        center = center / np.linalg.norm(center)  # Project to unit sphere
        dual_vertices.append(center)
    return np.array(dual_vertices)

def check_regularity(vertices: np.ndarray, edges: list) -> bool:
    """Check if polytope is regular (all edges same length)."""
    edge_lengths = []
    for edge in edges:
        length = np.linalg.norm(vertices[edge[0]] - vertices[edge[1]])
        edge_lengths.append(length)
    
    # Regular if all edge lengths are within 1% of mean
    mean_length = np.mean(edge_lengths)
    return all(abs(l - mean_length) < 0.01 * mean_length for l in edge_lengths)

# Test our functions
print("Testing polytope operations:\n")

for name in ['tetrahedron', 'octahedron', 'icosahedron']:
    vertices, edges = generate_platonic_solid(name)
    is_regular = check_regularity(vertices, edges)
    
    print(f"{name.capitalize()}:")
    print(f"  Vertices: {len(vertices)}")
    print(f"  Edges: {len(edges)}")
    print(f"  Regular: {is_regular}")
    print()

## The 24-Cell: When 3D Intuition Fails

The 24-cell exists only in 4D. It has no 3D analog. This shows the limits of our intuition!

In [ ]:
from src.polytope_core.twentyfour_cell import TwentyFourCell

def visualize_24cell_projections():
    """Show multiple projections of the 24-cell."""
    cell24 = TwentyFourCell()
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5), subplot_kw={'projection': '3d'})
    
    # Different projection methods
    projections = [
        ("Stereographic", lambda v: v[:3] / (1 - v[3] + 1e-10)),
        ("Orthogonal", lambda v: v[:3]),
        ("Schlegel", lambda v: v[:3] / (2 - v[3] + 1e-10))
    ]
    
    for ax, (name, proj_func) in zip(axes, projections):
        # Project 4D vertices to 3D
        vertices_3d = np.array([proj_func(v) for v in cell24.vertices])
        
        # Plot vertices
        ax.scatter(*vertices_3d.T, c='red', s=50, alpha=0.8)
        
        # Plot some edges (subset for clarity)
        for i, edge in enumerate(cell24.edges[:50]):  # First 50 edges
            v1, v2 = vertices_3d[edge[0]], vertices_3d[edge[1]]
            ax.plot3D(*np.array([v1, v2]).T, 'b-', alpha=0.3, linewidth=1)
        
        ax.set_title(f'{name} Projection\nof 4D 24-cell to 3D')
        ax.set_box_aspect([1,1,1])
    
    plt.suptitle('The 24-Cell: A 4D Polytope With No 3D Analog', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    print("\n24-Cell Properties:")
    print(f"• 24 vertices (all at equal distance from center)")
    print(f"• 96 edges (each vertex connects to 8 others)")
    print(f"• 96 triangular faces")
    print(f"• 24 octahedral cells")
    print("\nKey insight: The 24-cell is self-dual in 4D!")
    print("This means it equals its own dual - a property no 3D polytope has.")
    print("\nBiological speculation: Could the brain use 4D representations")
    print("that we only perceive as 3D 'shadows'?")

visualize_24cell_projections()

## Biological Examples Gallery

In [ ]:
def biological_polytope_gallery():
    """Show real biological structures that use polytope geometry."""
    
    examples = [
        {
            'name': 'Clathrin Cage',
            'polytope': 'truncated_icosahedron',
            'description': 'Protein coat for endocytosis',
            'vertices': 60,
            'why': 'Maximizes volume while minimizing protein'
        },
        {
            'name': 'Adenovirus Capsid',
            'polytope': 'icosahedron',
            'description': 'Viral protein shell',
            'vertices': 12,
            'why': 'Most efficient spherical packing'
        },
        {
            'name': 'Magnetosome Chain',
            'polytope': 'octahedron',
            'description': 'Magnetic navigation organelles',
            'vertices': 6,
            'why': 'Crystal structure of magnetite'
        },
        {
            'name': 'Diatom Frustule',
            'polytope': 'various',
            'description': 'Silica cell wall',
            'vertices': 'many',
            'why': 'Optimizes strength-to-weight ratio'
        }
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    axes = axes.flatten()
    
    for ax, example in zip(axes, examples):
        ax.text(0.5, 0.9, example['name'], 
               transform=ax.transAxes, ha='center', fontsize=14, weight='bold')
        ax.text(0.5, 0.8, example['description'], 
               transform=ax.transAxes, ha='center', fontsize=10)
        ax.text(0.5, 0.5, f"Polytope: {example['polytope']}", 
               transform=ax.transAxes, ha='center', fontsize=10)
        ax.text(0.5, 0.4, f"Vertices: {example['vertices']}", 
               transform=ax.transAxes, ha='center', fontsize=10)
        ax.text(0.5, 0.2, example['why'], 
               transform=ax.transAxes, ha='center', fontsize=10, style='italic')
        ax.axis('off')
        
        # Add simple polytope sketch
        if example['polytope'] == 'icosahedron':
            # Draw simple icosahedron projection
            angles = np.linspace(0, 2*np.pi, 6)
            x, y = 0.5 + 0.3*np.cos(angles), 0.65 + 0.3*np.sin(angles)
            ax.plot(x, y, 'b-', transform=ax.transAxes)
        elif example['polytope'] == 'octahedron':
            # Draw simple octahedron projection (square)
            square = plt.Rectangle((0.35, 0.5), 0.3, 0.3, 
                                 transform=ax.transAxes, 
                                 fill=False, edgecolor='blue')
            ax.add_patch(square)
    
    plt.suptitle('Polytopes in Biology: Real Examples', fontsize=16)
    plt.tight_layout()
    plt.show()

biological_polytope_gallery()

print("\nKey Observations:")
print("• Icosahedral symmetry dominates in viruses (20-60nm scale)")
print("• Octahedral/cubic symmetry in crystals and minerals")
print("• Hexagonal packing in 2D surfaces (retina, epithelia)")
print("• Tetrahedral coordination in neural microcircuits")
print("\nConclusion: Polytopes are not mathematical abstractions -")
print("they are the fundamental shapes biology uses for computation!")

## Exercises

Test your understanding with these problems:

In [ ]:
def create_exercises():
    """Interactive exercises with hidden solutions."""
    
    # Exercise 1: Identify polytope from vertex count
    print("Exercise 1: Identify the Platonic solid")
    print("A polytope has 20 vertices. Which Platonic solid is it?\n")
    
    answer1 = widgets.Text(placeholder='Type your answer...')
    check1 = widgets.Button(description='Check Answer')
    output1 = widgets.Output()
    
    def check_answer1(b):
        with output1:
            output1.clear_output()
            if answer1.value.lower() in ['dodecahedron', 'the dodecahedron']:
                print("✓ Correct! The dodecahedron has 20 vertices.")
                print("Remember: V-E+F=2, and it has 12 pentagonal faces.")
            else:
                print("✗ Try again. Hint: Which solid has pentagonal faces?")
    
    check1.on_click(check_answer1)
    display(answer1, check1, output1)
    
    print("\n" + "="*50 + "\n")
    
    # Exercise 2: Compute dual
    print("Exercise 2: Find the dual polytope")
    print("What is the dual of an icosahedron (20 faces, 12 vertices)?\n")
    
    answer2 = widgets.Text(placeholder='Type your answer...')
    check2 = widgets.Button(description='Check Answer')
    output2 = widgets.Output()
    
    def check_answer2(b):
        with output2:
            output2.clear_output()
            if answer2.value.lower() in ['dodecahedron', 'the dodecahedron']:
                print("✓ Correct! The dual swaps faces and vertices.")
                print("Icosahedron: 20 faces, 12 vertices")
                print("Dodecahedron: 12 faces, 20 vertices")
            else:
                print("✗ Try again. Hint: In duals, #faces ↔ #vertices")
    
    check2.on_click(check_answer2)
    display(answer2, check2, output2)
    
    print("\n" + "="*50 + "\n")
    
    # Exercise 3: Tiling pattern
    print("Exercise 3: Tiling the plane")
    print("How many regular hexagons meet at each vertex in a hexagonal tiling?\n")
    
    answer3 = widgets.IntSlider(value=4, min=2, max=6, description='Number:')
    check3 = widgets.Button(description='Check Answer')
    output3 = widgets.Output()
    
    def check_answer3(b):
        with output3:
            output3.clear_output()
            if answer3.value == 3:
                print("✓ Correct! Exactly 3 hexagons meet at each vertex.")
                print("Each hexagon has interior angle 120°")
                print("3 × 120° = 360° (perfect fit!)")
            else:
                print(f"✗ With {answer3.value} hexagons: {answer3.value}×120° = {answer3.value*120}°")
                print("This doesn't equal 360°, so it won't tile!")
    
    check3.on_click(check_answer3)
    display(answer3, check3, output3)
    
    print("\n" + "="*50 + "\n")
    
    # Exercise 4: Biological structure prediction
    print("Exercise 4: Predict the polytope")
    print("A spherical virus needs to minimize the amount of protein used.")
    print("Which Platonic solid should it use for its capsid?\n")
    
    options = ['Tetrahedron', 'Cube', 'Octahedron', 'Dodecahedron', 'Icosahedron']
    answer4 = widgets.Dropdown(options=options, description='Choice:')
    check4 = widgets.Button(description='Check Answer')
    output4 = widgets.Output()
    
    def check_answer4(b):
        with output4:
            output4.clear_output()
            if answer4.value == 'Icosahedron':
                print("✓ Correct! The icosahedron is the most sphere-like.")
                print("It maximizes volume while minimizing surface area.")
                print("This is why most spherical viruses use icosahedral symmetry.")
            else:
                print(f"✗ The {answer4.value} is not optimal.")
                print("Hint: Which polytope best approximates a sphere?")
    
    check4.on_click(check_answer4)
    display(answer4, check4, output4)

create_exercises()

## Synthesis: Polytopes ARE Biological Computation

We've discovered that polytopes are not human mathematical inventions but nature's fundamental computational shapes:

1. **Limited Options**: Only 5 Platonic solids exist in 3D - biology must use these
2. **Optimal Solutions**: Each polytope solves specific optimization problems
3. **Scale Invariance**: Same shapes appear from molecules to organisms
4. **Discrete-Continuous Bridge**: Polytopes naturally connect discrete and continuous
5. **Higher Dimensions**: 4D structures like the 24-cell may underlie neural computation

### Next Steps

In the following notebooks, we'll explore:
- How spherical harmonics on polytopes create biological shapes
- How the 24-cell encodes error correction in neural circuits
- How stereographic projection connects 3D structure to 2D perception

Remember: When you see hexagons in the retina, tetrahedra in neural circuits, or icosahedra in viruses, you're seeing nature computing with polytopes!